In [11]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, handoff, function_tool, RunContextWrapper
from pydantic import BaseModel

In [12]:
load_dotenv()

api_key=os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Problem with API key")

In [13]:
from agents import trace

class Issue(BaseModel):
    issue:str
    why:str # Why can you not handle it

@function_tool
def create_ticket(issue:str):
    print(f"creating ticket for the {issue}")
    return "Ticket created with ID. 12345"

def on_manager_handoff(ctx:RunContextWrapper[None], input:Issue):
    print(f"Escalating to manager : {input.issue}")
    print(f"Reason for escalation : {input.why}")


manager_agent=Agent(
    name="Manager Agent",
    handoff_description="You handle the complex issues that the Customer Service Agent is not able to resolve by itself ",
    instructions="You handle escalted customer issues that Customer Service Agent is not able to resolve by itself"
                  "You will receive the issue and the reason for escalation. If the issue cannot be resolved immediately, create a ticket",
    tools=[create_ticket]
)

customer_service_agent = Agent(
    name= "Customer Service Agent",
    instructions="You are a customer support agent which helps customer with basic trouble shooting and general queries"
                  "If you are unable to answer escalte to the manager, with reason why you cannot fix the issue yourself",
    handoffs=[handoff(agent= manager_agent, input_type=Issue, on_handoff=on_manager_handoff)]
)

with trace("Customer Hotline service"):
    issue = "I want a refund but your system wont let me process it. You website is just blank"
    result= await Runner.run(customer_service_agent, issue)

    print(result.final_output)

Escalating to manager : The customer is unable to process a refund due to a blank website page.
Reason for escalation : The system issue is beyond basic troubleshooting steps and requires technical support to fix the website functionality.
creating ticket for the The customer is unable to process a refund because the website is displaying as a blank page.
I've created a support ticket for your issue. Our technical team will look into the website problem as soon as possible. Your ticket ID is **12345**. We'll keep you updated!
